In [ ]:
from collections import namedtuple
import pathlib
import random

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project as op,
                    project_loader as opl,
                    reference_origins as origin)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
def get_project_monitors(proj):
    return proj.Monitors.Items

project_monitors = toolz.pipe(
    projects,
    toolz.valmap(get_project_monitors),
)
project_monitors

In [ ]:
toolz.pipe(project_monitors.values(),
           toolz.map(len),
           list,)

In [ ]:
def get_monitor_times(m):
    return m.StartTime, m.StopTime

project_monitor_times = toolz.pipe(
    project_monitors,
    toolz.valmap(toolz.map(get_monitor_times)),
    toolz.valmap(list),
)
project_monitor_times

In [ ]:
def utc_times(times):
    def utc_time(t):
        return t.ToString('o')

    return tuple(toolz.map(utc_time, times))

project_monitor_utc_times = toolz.pipe(
    project_monitor_times,
    toolz.valmap(toolz.map(utc_times)),
    toolz.valmap(list),
)
project_monitor_utc_times

In [ ]:
selected_field_name = 'bakken'

In [ ]:
def summarize_times(field_name, times):
    field_name_times = times[field_name]
    return {
        'field': field_name,
        'index': list(range(len(field_name_times))),
        'start': [ts[0] for ts in field_name_times],
        'stop': [ts[1] for ts in field_name_times],
    }

In [ ]:
pd.DataFrame(data=summarize_times('bakken', project_monitor_utc_times))

In [ ]:
pd.DataFrame(data=summarize_times('montney', project_monitor_utc_times))

In [ ]:
def merge_time_series(so_far, m):
    return toolz.assoc(so_far, m.DisplayName, m)

project_monitor_time_series = toolz.pipe(
    project_monitors,
    toolz.valmap(toolz.map(lambda m: m.TimeSeries)),
    toolz.valmap(list),
    # toolz.valmap(toolz.reduce(merge_time_series)),
)
project_monitor_time_series